# Criando uma Árvore de Decisão

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score




In [2]:
dados = pd.read_csv('data/creditcard.csv')
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### O número de: transação, fraudes e dados normais

In [3]:
dados['Class'].unique()

array([0, 1])

In [4]:
n_transacoes = dados['Class'].count()
n_fraudes = dados['Class'].sum()
n_normais = n_transacoes - n_fraudes

fraudes_porc = ((n_fraudes / n_transacoes) *100)
normais_porc = (n_normais / n_transacoes) *100


print("Número de transações: ", n_transacoes)
print("Número de fraudes: ", n_fraudes, ", ", "%.2f" %fraudes_porc, "%", sep = "")
print("Número de fraudes: ", n_normais, ", ", "%.2f" %normais_porc, "%", sep = "")

Número de transações:  284807
Número de fraudes: 492, 0.17%
Número de fraudes: 284315, 99.83%


In [5]:
x = dados.drop('Class', axis = 1).values
y = dados['Class'].values

## Funções

### Criando uma árvore de decisão com o scikit-learn

In [6]:
def executar_validador(x, y):
    # Vamos garantir que o meu conjunto vai ter 10%
    # das transações de cada uma das classes.
    validador = StratifiedShuffleSplit(n_splits=1,
                                   test_size=0.1, random_state=0)
    validador.split(x, y)
    for treino_id, teste_id in validador.split(x, y):
        x_train, x_test = x[treino_id], x[teste_id]
        y_train, y_test = y[treino_id], y[teste_id]
    return x_train, x_test, y_train, y_test

### Gerando o modelo 

In [7]:
def executar_classificador(classificador, x_train, x_test, y_train):
    arvore = classificador.fit(x_train, y_train)
    y_pred = arvore.predict(x_test)
    return y_pred

### Salvando uma imagem da árvore

In [8]:
def salvar_arvore(classificador, nome):
    plt.figure(figsize=(200,100))
    tree.plot_tree(classificador, filled=True, fontsize=14)
    plt.savefig(nome)
    plt.close()

### Medindo a acurácia da árvore

In [9]:
def validar_arvore(y_test, y_pred):
    print("accuracy_score: ", accuracy_score(y_test, y_pred))
    print("precision_score: ", precision_score(y_test, y_pred))
    print("recall_score: ", recall_score(y_test, y_pred))
    print("confusion_matrix: \n", confusion_matrix(y_test, y_pred))
    

## EXECUTANDO

### Execução do validador

In [10]:
x = dados.drop('Class', axis = 1).values
y = dados['Class'].values
x_train, x_test, y_train, y_test = executar_validador(x, y)

### Execução do classificador `DecisionTreeClassifier()`

In [11]:
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(
    classificador_arvore_decisao, x_train, x_test, y_train)

### Criando a figura da árvore de decisão

In [12]:
salvar_arvore(classificador_arvore_decisao, 'arvore_decisao1.png')

### validacao da árvore de decisão


In [13]:
validar_arvore(y_test, y_pred_arvore_decisao)

accuracy_score:  0.9989817773252344
precision_score:  0.7083333333333334
recall_score:  0.6938775510204082
confusion_matrix: 
 [[28418    14]
 [   15    34]]


<img src="data/Aula3_video2_imagem1.png">

### Verificando os parâmetros

In [14]:
print("Parâmetros: \n", classificador_arvore_decisao.get_params())

print("\nProfundidade:", classificador_arvore_decisao.get_depth())

Parâmetros: 
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}

Profundidade: 21


### Limitando a profundidade da árvore em 10

In [15]:
classificador_arvore_decisao_2 = tree.DecisionTreeClassifier(max_depth=10, 
                                                           random_state=0)
y_pred_arvore_decisao_2 = executar_classificador(
    classificador_arvore_decisao_2, x_train, x_test, y_train)

In [16]:
validar_arvore(y_test, y_pred_arvore_decisao_2)

accuracy_score:  0.9994733330992591
precision_score:  0.9473684210526315
recall_score:  0.7346938775510204
confusion_matrix: 
 [[28430     2]
 [   13    36]]


### Set o número mínimo de 10 instâncias em cada nó

In [17]:
classificador_arvore_decisao_3 = tree.DecisionTreeClassifier(max_depth=10,
                                                             min_samples_leaf=10,
                                                           random_state=0)
y_pred_arvore_decisao_3 = executar_classificador(
    classificador_arvore_decisao_3, x_train, x_test, y_train)

In [18]:
validar_arvore(y_test, y_pred_arvore_decisao_3)

accuracy_score:  0.9993679997191109
precision_score:  0.8604651162790697
recall_score:  0.7551020408163265
confusion_matrix: 
 [[28426     6]
 [   12    37]]


### Das características que vimos e dessas mudanças que fizemos, o que funcionou o melhor para nós e para nossa árvore foi determinar somente a profundidade máxima igual a 10. 